# Preprocesamiento de Datos
Limpieza, tokenización y vectorización de los textos.

In [ ]:
import os
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Rutas
raw_path       = os.path.join("..", "data", "raw", "D57000_complete.csv")
processed_dir  = os.path.join("..", "data", "processed")
split_dir      = os.path.join("..", "data", "split")
vectorizer_dir = os.path.join("..", "models")
vectorizer_path = os.path.join(vectorizer_dir, "vectorizer.pkl")

# Leer archivo con separador adecuado
df = pd.read_csv(raw_path, sep=";", encoding="utf-8", encoding_errors="replace")

# Verificamos columnas: ['ID', 'Label', 'Titulo', 'Descripcion', 'Fecha']
print("🔹 Columnas:", df.columns.tolist())

# Combinamos Titulo y Descripcion como texto
df["text"] = df["Titulo"].fillna("") + ". " + df["Descripcion"].fillna("")

# Mantenemos solo texto y etiqueta
df = df[["text", "Label"]].rename(columns={"Label": "label"})

# Limpieza del texto
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r"http\S+", "", texto)  # quitar URLs
    texto = re.sub(r"[^a-záéíóúñü\s]", "", texto)  # solo letras y espacios
    texto = re.sub(r"\s+", " ", texto).strip()
    return texto

df["text"] = df["text"].fillna("").apply(limpiar_texto)

# Eliminar stopwords en español
stopwords_es = set(stopwords.words('spanish'))

def eliminar_stopwords(texto):
    return " ".join([palabra for palabra in texto.split() if palabra not in stopwords_es])

df["text"] = df["text"].apply(eliminar_stopwords)

# Guardar versión preprocesada
os.makedirs(processed_dir, exist_ok=True)
df.to_csv(os.path.join(processed_dir, "datos_preprocesados.csv"), index=False)

# Vectorización con TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df["text"])
y = df["label"]

# Guardar vectorizador
os.makedirs(vectorizer_dir, exist_ok=True)
joblib.dump(vectorizer, vectorizer_path)

# Separar en entrenamiento y prueba (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Guardar como archivos serializados
os.makedirs(split_dir, exist_ok=True)
joblib.dump((X_train, y_train), os.path.join(split_dir, "train.pkl"))
joblib.dump((X_test, y_test), os.path.join(split_dir, "test.pkl"))

print("✅ Preprocesamiento y vectorización completados.")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


🔹 Columnas: ['ID', 'Label', 'Titulo', 'Descripcion', 'Fecha']
✅ Preprocesamiento y vectorización completados.
